In [ ]:
import pandas as pd
import numpy as np
from numpy._core.defchararray import count
import matplotlib.pyplot as plt
from scipy import stats
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import random

In [ ]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

df_pesticides2 = pd.read_csv('Datasets/historical_data_2000_2022_filtered.csv')
df_confounders = pd.read_csv('Datasets/copd_aqi_poverty_demographics.csv')
df_population = pd.read_csv('Datasets/Population_Census_Numbers_2000_2025.csv')


print(f'Pesticides: {df_pesticides2.columns.tolist()}')
print(f'Confounders: {df_confounders.columns.tolist()}')

print(f'\nPesticide shape: {df_pesticides2.shape}')
print(f'Confounders shape: {df_confounders.shape}')

print(f'\nPesticide yrs: {df_pesticides2['YEAR'].min()}--{df_pesticides2['YEAR'].max()} ')
print(f'Confounders yrs: {df_confounders['Year'].min()}--{df_confounders['Year'].max()}')

In [ ]:
#Data Standardization

df_pesticides2.columns = map(str.lower, df_pesticides2.columns)
df_confounders.columns = map(str.lower, df_confounders.columns)
df_pesticides2 = df_pesticides2.rename(columns={'county_name': 'county', 'total_lbs_ai': 'total_pesticide_lbs'})
df_confounders = df_confounders.rename(columns={'counties': 'county'})
df_pesticides2['county'] = df_pesticides2['county'].replace('Tuolomne','Tuolumne')
df_confounders['county'] = df_confounders['county'].replace('Tuolomne','Tuolumne')
df_pesticides2['county'] = df_pesticides2['county'].str.lower()
df_confounders['county'] = df_confounders['county'].str.lower()
df_pesticides2['county'] = df_pesticides2['county'].str.strip()
df_confounders['county'] = df_confounders['county'].str.strip()




df_confounders['median aqi'] = df_confounders.groupby('county')['median aqi'].transform(
    lambda x: x.fillna(x.mean())
)
df_confounders['median aqi'] = df_confounders['median aqi'].fillna(
    df_confounders['median aqi'].mean()
)



pesticide_counties = set(df_pesticides2['county'].unique())
confounding_counties = set(df_confounders['county'].unique())
county_assessment = pesticide_counties - confounding_counties

print(f'\nCounties not in one or the other: {county_assessment}')
print(f'\nCounties to be excluded: {county_assessment}')
print(f'Overlapping counties: {len(pesticide_counties & confounding_counties)}')
print(f'\nPesticides: {df_pesticides2.columns.tolist()}')
print(f'Confounders: {df_confounders.columns.tolist()}')
print(f'\nNulls for pesticide data: {df_pesticides2.isnull().sum()}')
print(f'\nNulls for confounders data: {df_confounders.isnull().sum()}')


In [ ]:
#Population Data
df_population = df_population.rename(columns={'County': 'county'})

population_long = df_population.melt(
    id_vars=['county'],
    var_name='date_str',
    value_name='total_population'
)

def year_rewrite(date_str):
  year_num = int(date_str.split('/')[-1])

  if year_num >= 1900:
    return year_num
  elif year_num <= 25:
    return 2000 + year_num
  else:
    return 1900 + year_num

population_long['year'] = population_long['date_str'].apply(year_rewrite)

population_long2 = population_long.drop(columns=['date_str'])

population_long2['total_population'] = population_long2['total_population'].str.replace('\t','')
population_long2['total_population'] = population_long2['total_population'].str.replace(',','')
population_long2['total_population'] = population_long2['total_population'].astype(float)
population_long2['county'] = population_long2['county'].str.lower()
population_long2['county'] = population_long2['county'].str.strip()


print(f'\nPopulation shape: {population_long2.shape}')
print('\nCounties:', population_long2['county'].nunique())
population_long2.head(11)

In [ ]:
# Aggregating Pesticide Data by County-Year

rows_by_county_year = df_pesticides2.groupby(['year','county']).size().head(6)
print(rows_by_county_year)

df_pesticides2 = df_pesticides2[(df_pesticides2['year'] >= 2000) & (df_pesticides2['year'] <= 2022)]
df_confounders = df_confounders[(df_confounders['year'] >= 2000) & (df_confounders['year'] <= 2022)]


pesticide_overall = df_pesticides2.groupby(['county', 'year']).agg({
    'total_pesticide_lbs': 'sum',
    'total_acres_treated': 'sum'
}).reset_index()



print(f'Total observations: {len(pesticide_overall)}')
print(f'Shape: {pesticide_overall.shape}')
print(f'Year range: {pesticide_overall['year'].min()} - {pesticide_overall['year'].max()}')
print(f'Counties: {pesticide_overall['county'].nunique()}')
print('\nFirst 10 rows:')
print(pesticide_overall.head(10))


In [ ]:
#Normalizing Pesticide data by population

population_long2 = population_long2[(population_long2['year'] >= 2000) & (population_long2['year'] <= 2022)]

pesticide_overall_merged = pesticide_overall.merge(population_long2,on=['county','year'],how='left')

pesticide_overall_merged['total_pesticide_lbs_per_100k'] = ((pesticide_overall_merged['total_pesticide_lbs']/pesticide_overall_merged['total_population'])*100000)
pesticide_overall_merged['total_acres_treated_per_100k'] = ((pesticide_overall_merged['total_acres_treated']/pesticide_overall_merged['total_population'])*100000)


pesticide_overall_merged_clean = pesticide_overall_merged.dropna(subset=['total_population'])
print(f'Rows dropped: {len(pesticide_overall_merged) - len(pesticide_overall_merged_clean)}')

print('\nFinal summary statistics:')
print(pesticide_overall_merged_clean[['total_pesticide_lbs_per_100k', 'total_acres_treated_per_100k']].describe())
print(pesticide_overall_merged_clean.head(10))
pesticide_overall_merged = pesticide_overall_merged_clean


In [ ]:
#Creating Lag Features

def lag_features(df, column, lag_years=[1, 2, 3, 5,10,15,20]):
  df_merged_sorted = pesticide_overall_merged.sort_values(['county','year']).copy()

  for col in column:
    for lag in lag_years:
      df_merged_sorted[f'{col}_lag{lag}'] = df_merged_sorted.groupby('county')[col].shift(lag)
  return df_merged_sorted

to_lag = ['total_pesticide_lbs_per_100k','total_acres_treated_per_100k']

pesticides_with_lags = lag_features(pesticide_overall_merged, to_lag, lag_years=[1, 2, 3, 5, 10, 15, 20])


print(f'Original columns: {len(pesticide_overall_merged.columns)}')
print(f'Columns with lags: {len(pesticides_with_lags.columns)}')
print(f'New lag columns added: {len(pesticides_with_lags.columns) - len(pesticide_overall_merged.columns)}')

print(f'All columns: {pesticides_with_lags.columns.to_list()}')

pesticides_with_lags.head(6)


In [ ]:
#Creating cumulative Exposure Features

def cumulative_features(df, column, windows=[3,5,10,15,20]):
  df_copied = df.copy()

  for col in column:
    for window in windows:
      df_copied[f'{col}_cumulative_sum{window}year'] = (df_copied.groupby('county')[col].rolling(window=window, min_periods=1).sum().reset_index(level=0, drop=True))
      df_copied[f'{col}_cumulative_mean{window}year'] = (df_copied.groupby('county')[col].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True))
  return df_copied


to_accumulate = ['total_pesticide_lbs_per_100k','total_acres_treated_per_100k']

final_pesticide = cumulative_features(pesticides_with_lags, to_accumulate, windows=[3,5,10,15,20])


print(f'Columns before: {len(pesticides_with_lags.columns)}')
print(f'Columns after: {len(final_pesticide.columns)}')
print(f'New features added: {len(final_pesticide.columns) - len(pesticides_with_lags.columns)}')

final_pesticide.head(6)


In [ ]:
# Merging Pesticide Features with Confounders

final_merged_with_confounders = df_confounders.merge(final_pesticide,on=['county','year'],how='inner')
complete_dataset = final_merged_with_confounders


county_coverage = complete_dataset.groupby('county')['year'].agg(['count', 'min', 'max'])
county_coverage.columns = ['num_years', 'first_year', 'last_year']
county_coverage = county_coverage.sort_values('num_years')

print(f'County Coverage Summary:{county_coverage}')
print(f'\nCounties with complete data (23 years): {(county_coverage['num_years'] == 23).sum()}')
print(f'Counties with partial data: {(county_coverage['num_years'] < 23).sum()}')






In [ ]:
# Handling missing values

complete_dataset_filtered = complete_dataset[complete_dataset['year'] >= 2005].copy()


lag10_cols = [col for col in complete_dataset_filtered.columns if 'lag10' in col]
lag15_cols = [col for col in complete_dataset_filtered.columns if 'lag15' in col]
lag20_cols = [col for col in complete_dataset_filtered.columns if 'lag20' in col]
complete_dataset_filtered = complete_dataset_filtered.drop(columns=lag20_cols)
complete_dataset_filtered = complete_dataset_filtered.drop(columns=lag15_cols)
complete_dataset_filtered = complete_dataset_filtered.drop(columns=lag10_cols)


complete_dataset_cleaned = complete_dataset_filtered.dropna()
print('Cleaned Dataset summarization')
print(f'- Shape: {complete_dataset_cleaned.shape}')
print(f'- Year range: {complete_dataset_cleaned['year'].min()} - {complete_dataset_cleaned['year'].max()}')
print(f'- Counties: {complete_dataset_cleaned['county'].nunique()}')
print(f'- Total features: {complete_dataset_cleaned.shape[1] - 3}')
print(f'- Observations per feature: {len(complete_dataset_cleaned) / (complete_dataset_cleaned.shape[1] - 3):.1f}')
print(complete_dataset_cleaned['copd_hospitalization_rate'].describe())




In [ ]:
#Validating Lag features

county_counts = final_pesticide.groupby('county').size()
sample_county = county_counts.idxmax()

validation_sample = final_pesticide[final_pesticide['county'] == sample_county].sort_values('year')

display_cols = ['year', 'total_pesticide_lbs_per_100k',
                'total_pesticide_lbs_per_100k_lag1',
                'total_pesticide_lbs_per_100k_lag5',
                'total_pesticide_lbs_per_100k_lag10',
                'total_pesticide_lbs_per_100k_lag20']

print(validation_sample[display_cols].head(25).to_string(index=False))

In [ ]:
# Feature Organization

indentifiers = ['county','years']
target = 'copd_hospitalization_rate'

confounding_features = [
    'median aqi',
    'pct_under_18', 'pct_18_64', 'pct_65_plus', 'median_age',
    'pct_ai/an', 'pct_asian', 'pct_black', 'pct_latino',
    'pct_multi_race', 'pct_nh/pi', 'pct_white',
    'poverty_allages_percent_est', 'median_household_income_est'
]
pesticide_features = ['total_pesticide_lbs', 'total_acres_treated',
                 'total_population', 'total_pesticide_lbs_per_100k',
                 'total_acres_treated_per_100k']

pesticide_lag = [col for col in complete_dataset_cleaned.columns if 'lag' in col]
pesticide_cumulative = [col for col in complete_dataset_cleaned.columns if 'cumulative' in col]


features_to_drop = []
sum_features = [col for col in complete_dataset_cleaned.columns if 'cumulative_sum' in col]
features_to_drop.extend(sum_features)
lag_to_drop = [col for col in complete_dataset_cleaned.columns if 'lag3' in col or 'lag5' in col]
features_to_drop.extend(lag_to_drop)
cum_mean_to_drop = [col for col in complete_dataset_cleaned.columns
                    if any(x in col for x in ['cumulative_mean3year',
                                               'cumulative_mean10year',
                                               'cumulative_mean15year'])]
features_to_drop.extend(cum_mean_to_drop)
raw_to_drop = ['total_pesticide_lbs', 'total_acres_treated']
features_to_drop.extend(raw_to_drop)
complete_dataset_final = complete_dataset_cleaned.drop(columns=features_to_drop)


print(f'Remaining: {[col for col in complete_dataset_final.columns if any(x in col for x in ['pesticide', 'acres', 'population'])]}')


In [ ]:
# Data prep for XGBoost

pesticide_features = ['total_population', 'total_pesticide_lbs_per_100k',
                      'total_acres_treated_per_100k', 'total_pesticide_lbs_per_100k_lag1',
                      'total_pesticide_lbs_per_100k_lag2', 'total_acres_treated_per_100k_lag1',
                      'total_acres_treated_per_100k_lag2',
                      'total_pesticide_lbs_per_100k_cumulative_mean5year',
                      'total_pesticide_lbs_per_100k_cumulative_mean20year',
                      'total_acres_treated_per_100k_cumulative_mean5year',
                      'total_acres_treated_per_100k_cumulative_mean20year']

confounders = ['median aqi', 'pct_under_18', 'pct_18_64', 'pct_65_plus', 'median_age',
               'pct_ai/an', 'pct_asian', 'pct_black', 'pct_latino', 'pct_multi_race',
               'pct_nh/pi', 'pct_white', 'poverty_allages_percent_est',
               'median_household_income_est']

all_features = pesticide_features + confounders

X = complete_dataset_final[all_features]
Y = complete_dataset_final['copd_hospitalization_rate']

print(f'X shape: {X.shape}')
print(f'Y shape: {Y.shape}')

train_df, test_df = train_test_split(
    complete_dataset_final,
    test_size=0.2,
    random_state=42
)


X_train = train_df[all_features].values
y_train = train_df['copd_hospitalization_rate'].values

X_test = test_df[all_features].values
y_test = test_df['copd_hospitalization_rate'].values

print(f'\nTarget variable (COPD rate) statistics:')
print(f' Train - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}, Range: [{y_train.min():.2f}, {y_train.max():.2f}]')
print(f' Test  - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}, Range: [{y_test.min():.2f}, {y_test.max():.2f}]')


In [ ]:
# Baseline XGBoost Model Training

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.05,
    max_depth=3,
    n_estimators=50,
    min_child_weight=10,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=1.0,
    reg_lambda=1.0,
    random_state=42
)


model.fit(X_train,y_train, verbose=False)
y_train_prediction = model.predict(X_train)
y_test_prediction = model.predict(X_test)

rmse_training = np.sqrt(mean_squared_error(y_train,y_train_prediction))
mae_training = (mean_absolute_error(y_train,y_train_prediction))
r2_score_training = r2_score(y_train,y_train_prediction)

rmse_testing = np.sqrt(mean_squared_error(y_test,y_test_prediction))
mae_testing = mean_absolute_error(y_test,y_test_prediction)
r2_score_testing = r2_score(y_test,y_test_prediction)


r2_diff = r2_score_training - r2_score_testing
print(f'Train R² - Test R² = {r2_diff:.4f}')

if r2_diff > 0.2:
  print(f'\nOverfitting Detected (R² gap: {r2_diff:.4f})')
elif r2_diff > 0.1:
  print(f'Moderate Overfitting (R² gap: {r2_diff:.4f})')
else:
  print(f'Minimal overfitting (R² gap: {r2_diff:.4f})')


naive_rmse = np.sqrt(mean_squared_error(y_test, [y_train.mean()]*len(y_test)))
improvement = (naive_rmse - rmse_testing) / naive_rmse * 100

print(f'\nBaseline comparison:')
print(f'Naive (predict train mean): RMSE ={naive_rmse:.2f}')
print(f'XGBoost: RMSE = {rmse_testing:.2f}')
print(f'Improvement: {improvement:.1f}%')

print('\nTraining set performance:')
print(f' RMSE: {rmse_training:.2f}')
print(f' MAE:  {mae_training:.2f}')
print(f' R²: {r2_score_training:.4f}')

print('\nTest set performance:')
print(f' RMSE: {rmse_testing:.2f}')
print(f' MAE: {mae_testing:.2f}')
print(f' R²: {r2_score_testing:.4f}')

print(f'\nModel Performance Summary:')
if r2_score_testing < 0:
    print(f'Test RÂ² is negative ({r2_score_testing:.4f})')
    print(f'Model performs worse than predicting the mean')
    print(f'Cause: Distribution shift between train/test')
elif r2_score_testing < 0.3:
    print(f'Test R² is low ({r2_score_testing:.4f})')
    print(f'Weak predictive power')
else:
    print(f'Test R² is {r2_score_testing:.4f}')
    print(f'Explains {r2_score_testing*100:.1f}% of variance')




In [ ]:
# Feature Importance

feat_importance = model.feature_importances_
df_importance = pd.DataFrame({'feature': all_features, 'importance': feat_importance}).sort_values('importance', ascending=False)

for idx, row in df_importance.head(15).iterrows():
    if row['feature'] in pesticide_features:
        category = '[PESTICIDE]'
    else:
        category = '[CONFOUNDER]'

    print(f'{row['feature']:50s} {category:13s} {row['importance']:.4f}')

pesticide_importance = df_importance[df_importance['feature'].isin(pesticide_features)]['importance'].sum()
confounder_importance = df_importance[df_importance['feature'].isin(confounders)]['importance'].sum()
total = pesticide_importance + confounder_importance


print(f'Pesticide features:  {pesticide_importance:.3f} ({pesticide_importance/total*100:.1f}%)')
print(f'Confounder features: {confounder_importance:.3f} ({confounder_importance/total*100:.1f}%)')

print('\nPesticide contribution to COPD prediction:')
if pesticide_importance/total > 0.30:
  print(f'Strong evidence: Pesticides account for {pesticide_importance/total*100:.1f}% of predictive power')
elif pesticide_importance/total > 0.15:
  print(f'Moderate evidence: Pesticides account for {pesticide_importance/total*100:.1f}% of predictive power')
elif pesticide_importance/total > 0.05:
  print(f'Weak evidence: Pesticides account for {pesticide_importance/total*100:.1f}% of predictive power')
